In [32]:
# Libs
import pandas as pd
import random

In [3]:
# General
SEED = 42
random.seed(SEED)

# Loading Data

## Keypoints

In [7]:
df_kp = pd.read_feather("../Data/Cleaned/df_dta_kp_cleaned.feather")
print(df_kp.head())

   ID                                          KEY_POINT
0  11  Patients reported uncertainty about whether sy...
1  12  Intense fatigue and a permanent state of weakn...
2  13          Need to spend long hours in the restroom.
3  21          Need to spend long hours in the restroom.
4  27          Need to spend long hours in the restroom.


## Questionnaire

In [31]:
df_kp_answers = pd.read_excel("../Data/Questionnaires_answers/Questionnaire_kp_answers.xlsx", engine='openpyxl')

df_kp_answers = df_kp_answers.T
df_kp_answers.columns = df_kp_answers.iloc[0].astype(str)
df_kp_answers = df_kp_answers[1:]
df_kp_answers.reset_index(inplace=True)
df_kp_answers.rename(columns={'index': 'Question'}, inplace=True)

print(df_kp_answers.iloc[2])

Tijdstempel
Question                      3/20\n\nKeypoint 1 (K1):\nStopping treatment w...
2025-04-28 10:32:43.897000                                                    2
2025-04-28 11:02:35.679000                                                    2
2025-04-30 11:17:45.719000                                                    3
Name: 2, dtype: object


# Data Cleaning

## Keypoints

## Questionnaire

# Pairwise comparison to score